# [Cupoy 機器學習百日馬拉松-期末考](https://www.kaggle.com/c/ml100marathon-final-exam/)
* Version
  * V1: 使用 MobileNet 做 transfer learning
  * V2: 使用 EarlyStopping/ReduceLROnPlateau，在 training 時依 val_loss 的情況來調整 learning rate.      
  * V3: 調整最後的 FC layer，由二層調整為三層 (神經元個數也調大了)
  * V4: 調整ImageDataGenerator 的參數，做了 rotate 與平移的設定
  * V5: 改用 VGG16 做 transfer learning
* Score  

| Version | Private Score |Public Score |  
|------|------|------|
|   V1  | 0.82600 | 0.83900 |
|   V2  | 0.91600 | 0.91900 |
|   V3  | 0.92200 | 0.92100 |
|   V4  | 0.92500 | 0.94500 |

## Version: V5

In [1]:
import os, shutil

In [2]:
img_base = '/home/ryanyao/docker_mount/data/ml100marathon-final-exam'
train_path = os.path.join(img_base, 'train')
test_path = os.path.join(img_base, 'test')
sub_train_path = os.path.join(img_base, 'sub_train')
sub_valid_path = os.path.join(img_base, 'sub_valid')
sub_test_path = os.path.join(img_base, 'sub_test')

In [3]:
class_set = os.listdir(train_path)
print(class_set)

['dandelion', 'tulip', 'rose', 'sunflower', 'daisy']


In [4]:
# Sanity checks
for i in class_set:
    print('train/%s/ images:' %(i), len(os.listdir(os.path.join(train_path, i))))

train/dandelion/ images: 687
train/tulip/ images: 633
train/rose/ images: 515
train/sunflower/ images: 488
train/daisy/ images: 500


In [5]:
# copy 各50張圖片到 sub_valid/sub_test, 其餘的 copy 到 sub_train
def generate_dir():
    for i_step in (sub_train_path, sub_valid_path, sub_test_path):
        os.mkdir(i_step)
        
    for i_step in (sub_train_path, sub_valid_path, sub_test_path):
        for i_class in class_set:
            os.mkdir(os.path.join(i_step, i_class))
            fnames = os.listdir(os.path.join(train_path, i_class))
            
            # sub_train dir
            if (i_step == sub_train_path):
                sub_fnames = fnames[:-100]
            elif (i_step == sub_valid_path):
                sub_fnames = fnames[-100:-50]
            else:
                sub_fnames = fnames[-50:]
            for fname in sub_fnames:
                src = os.path.join(train_path, i_class, fname)
                det = os.path.join(i_step, i_class, fname)
                # print(src)
                # print(det)
                shutil.copyfile(src, det)

# generate_dir()

In [6]:
import numpy as np
import keras
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
# from keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [7]:
# Workaround Issue: "Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR"
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [8]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3)

In [9]:
batch_size = 32
sub_train_batches = datagen.flow_from_directory(sub_train_path, target_size=(256,256), batch_size=batch_size)
sub_valid_batches = datagen.flow_from_directory(sub_valid_path, target_size=(256,256), batch_size=batch_size)
sub_test_batches = datagen.flow_from_directory(sub_test_path, target_size=(256,256), batch_size=batch_size)

Found 2323 images belonging to 5 classes.
Found 250 images belonging to 5 classes.
Found 250 images belonging to 5 classes.


In [10]:
# 檢視 image input size
print(sub_train_batches.image_shape)

(256, 256, 3)


### 下載 Keras NobileNet model 進行 transfer learning

In [12]:
#imports the mobilenet model and discards the last 1000 neuron layer.
# base_model = MobileNet(include_top=False, weights='imagenet') 
base_model = VGG16(weights='imagenet', include_top=False, input_shape = (256, 256, 3))
base_model.summary()

# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in base_model.layers[:5]:
    layer.trainable = False

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [13]:
x = base_model.output
# we add dense layers so that the model can learn more complex functions and classify for better results.
x = Flatten()(x)
x = Dense(1024, activation='relu')(x) 
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
preds = Dense(5, activation='softmax')(x) # final layer with softmax activation

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
model = Model(inputs=base_model.input, outputs=preds)

In [15]:
# # Check the model architecture
# for i,layer in enumerate(model.layers):
#     print(i, layer.name, layer.trainable)

In [16]:
# 輸出整個網路結構
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

### 使用 EarlyStopping/ReduceLROnPlateau，在 training 時依 val_loss 的情況來調整 learning rate.

In [17]:
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor="val_loss", patience=10, verbose=1)

from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(factor=0.3,min_lr=1e-12, monitor='val_loss', patience=5, verbose=1)

In [18]:
model.compile(optimizer = SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_sub_train = sub_train_batches.n // sub_train_batches.batch_size
step_size_sub_valid = sub_valid_batches.n // sub_valid_batches.batch_size
history = model.fit_generator(generator=sub_train_batches, 
                    steps_per_epoch=step_size_sub_train, 
                    validation_data=sub_valid_batches,
                    validation_steps=step_size_sub_valid,
                    epochs=50,
                    callbacks=[earlystop,reduce_lr],
                    verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
 - 57s - loss: 1.6113 - acc: 0.2795 - val_loss: 1.4749 - val_acc: 0.3705
Epoch 2/50
 - 51s - loss: 1.3522 - acc: 0.4442 - val_loss: 1.1991 - val_acc: 0.4541
Epoch 3/50
 - 45s - loss: 1.1115 - acc: 0.5513 - val_loss: 0.9564 - val_acc: 0.5780
Epoch 4/50
 - 46s - loss: 0.8843 - acc: 0.6496 - val_loss: 0.6725 - val_acc: 0.7615
Epoch 5/50
 - 48s - loss: 0.7814 - acc: 0.6995 - val_loss: 0.6791 - val_acc: 0.7018
Epoch 6/50
 - 46s - loss: 0.6535 - acc: 0.7566 - val_loss: 0.5421 - val_acc: 0.8073
Epoch 7/50
 - 47s - loss: 0.5859 - acc: 0.7811 - val_loss: 0.4320 - val_acc: 0.8394
Epoch 8/50
 - 46s - loss: 0.5326 - acc: 0.8042 - val_loss: 0.4640 - val_acc: 0.8165
Epoch 9/50
 - 47s - loss: 0.5166 - acc: 0.8091 - val_loss: 0.4613 - val_acc: 0.8438
Epoch 10/50
 - 39s - loss: 0.5028 - acc: 0.8089 - val_loss: 0.3692 - val_acc: 0.8670
Epoch 11/50
 - 41s - loss: 0.4736 - acc: 0.8321 - val_loss: 0.3622 - val_acc: 0.8716
Epoch 12/50
 - 36s - loss:

In [19]:
step_size_sub_test = sub_test_batches.n // sub_test_batches.batch_size
model.evaluate_generator(generator=sub_test_batches, steps=step_size_sub_test, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=2)

[0.5149751326867512, 0.8660714285714286]

In [20]:
model.metrics_names

['loss', 'acc']

### 上傳 Kaggle

In [21]:
print(sub_train_batches.class_indices)


{'daisy': 0, 'dandelion': 1, 'rose': 2, 'sunflower': 3, 'tulip': 4}


In [22]:
from keras.preprocessing import image
import pandas as pd

In [23]:
submit = pd.DataFrame(columns=['id', 'flower_class'])
fnames = os.listdir(test_path)

for idx, fname in enumerate(fnames):
    # Loads an image into PIL format.
    img = image.load_img(os.path.join(test_path, fname), target_size=(256, 256))
    
    # Converts a PIL Image instance to a Numpy array. shape=(224, 224, 3)
    img_data = image.img_to_array(img) 
    img_data = img_data/255.0
    
    # Insert a new axis that will appear at the `axis` position in the expanded array shape. shape=(1, 224, 224, 3)
    img_data = np.expand_dims(img_data, axis=0)                

    preds = model.predict(img_data)
    submit.loc[idx] = [fname.split('.')[0], preds.argmax()]

submit.to_csv('Day101_submission_v5.csv', index=False)